# Information Theory Measures w/ RBIG

In [1]:
import sys, os
from pyprojroot import here

# spyder up to find the root
root = here(project_files=[".here"])

# append to path
sys.path.append(str(root))

%load_ext autoreload
%autoreload 2

In [2]:
import sys

import numpy as np
from sklearn.utils import check_random_state
import matplotlib.pyplot as plt
import corner

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


---
## Total Correlation

In [3]:
#Parameters
n_samples = 10000
d_dimensions = 10

seed = 123

rng = check_random_state(seed)

#### Sample Data

In [4]:
# Generate random normal data
data_original = rng.randn(n_samples, d_dimensions)

# Generate random Data
A = rng.rand(d_dimensions, d_dimensions)

data = data_original @ A

# covariance matrix
C = A.T @ A
vv = np.diag(C)

#### Calculate Total Correlation

In [5]:
tc_original = np.log(np.sqrt(vv)).sum() - 0.5 * np.log(np.linalg.det(C))

print(f"TC: {tc_original:.4f}")

TC: 9.9326


### RBIG - TC

In [6]:
from rbig._src.total_corr import rbig_total_corr

In [7]:
tc_rbig = rbig_total_corr(data, zero_tolerance=30)

Completed! (Total Info Red: 9.9040):   3%|▎         | 31/1000 [00:02<01:16, 12.67it/s]


In [8]:
print(f"TC (RBIG): {tc_rbig:.4f}")
print(f"TC: {tc_original:.4f}")

TC (RBIG): 9.9040
TC: 9.9326


---

## Entropy

#### Sample Data

In [9]:
#Parameters
n_samples = 5000
d_dimensions = 10

seed = 123

rng = check_random_state(seed)

# Generate random normal data
data_original = rng.randn(n_samples, d_dimensions)

# Generate random Data
A = rng.rand(d_dimensions, d_dimensions)

data = data_original @ A


#### Calculate Entropy

In [10]:
from rbig._src.entropy import entropy_marginal

In [11]:
Hx = entropy_marginal(data)

H_original = Hx.sum() + np.linalg.slogdet(A)[1]

print(f"H: {H_original:.4f}")

H: 16.4634


In [12]:
print(f"H: {H_original * np.log(2):.4f}")

H: 11.4116


### Entropy RBIG

In [14]:
from rbig._src.entropy import entropy_rbig

In [16]:
H_rbig = entropy_rbig(data, zero_tolerance=30)

Completed! (Total Info Red: 9.2054):   3%|▎         | 31/1000 [00:01<00:59, 16.29it/s]


In [17]:
print(f"Entropy (RBIG): {H_rbig:.4f}")
print(f"Entropy: {H_original:.4f}")

Entropy (RBIG): 10.7413
Entropy: 16.4634


---
## Mutual Information

#### Sample Data

In [18]:
#Parameters
n_samples = 10000
d_dimensions = 10

seed = 123

rng = check_random_state(seed)

# Generate random Data
A = rng.rand(2 * d_dimensions, 2 * d_dimensions)

# Covariance Matrix
C = A @ A.T
mu = np.zeros((2 * d_dimensions))

dat_all = rng.multivariate_normal(mu, C, n_samples)

CX = C[:d_dimensions, :d_dimensions]
CY = C[d_dimensions:, d_dimensions:]

X = dat_all[:, :d_dimensions]
Y = dat_all[:, d_dimensions:]

#### Calculate Mutual Information

In [30]:
H_X = 0.5 * np.log(2 * np.pi * np.exp(1) *  np.abs(np.linalg.det(CX)))
H_Y = 0.5 * np.log(2 * np.pi * np.exp(1) * np.abs(np.linalg.det(CY)))
H = 0.5 * np.log(2 * np.pi * np.exp(1) * np.abs(np.linalg.det(C)))

mi_original = H_X + H_Y - H
mi_original #*= np.log(2)

print(f"MI: {mi_original:.4f}")

MI: 11.6444


### RBIG - Mutual Information

In [22]:
from rbig._src.mutual_info import MutualInfoRBIG

In [35]:
# Initialize RBIG class
rbig_model = MutualInfoRBIG(zero_tolerance=10)

# fit model to the data
rbig_model.fit(X, Y);

Completed! (Total Info Red: 9.5033):   1%|          | 11/1000 [00:01<02:41,  6.12it/s]


In [36]:
H_rbig = rbig_model.mutual_info()

print(f"MI (RBIG): {H_rbig:.4f}")
print(f"MI: {mi_original:.4f}")

MI (RBIG): 9.5033
MI: 11.6444


In [37]:
print(f"TC (X) (RBIG): {rbig_model.rbig_model_X.total_correlation():.4f}")
print(f"TC (Y) (RBIG): {rbig_model.rbig_model_Y.total_correlation():.4f}")
print(f"TC (XY_t) (RBIG): {rbig_model.rbig_model_XY.total_correlation():.4f}")

TC (X) (RBIG): 6.9717
TC (Y) (RBIG): 6.4171
TC (XY_t) (RBIG): 9.5033


---
## Kullback-Leibler Divergence (KLD)

#### Sample Data

In [15]:
#Parameters
n_samples = 10000
d_dimensions = 10
mu = 0.4          # how different the distributions are

seed = 123

rng = check_random_state(seed)

# Generate random Data
A = rng.rand(d_dimensions, d_dimensions)

# covariance matrix
cov = A @ A.T

# Normalize cov mat
cov = A / A.max()

# create covariance matrices for x and y
cov_x = np.eye(d_dimensions)
cov_y = cov_x.copy()

mu_x = np.zeros(d_dimensions) + mu
mu_y = np.zeros(d_dimensions)

# generate multivariate gaussian data
X = rng.multivariate_normal(mu_x, cov_x, n_samples)
Y = rng.multivariate_normal(mu_y, cov_y, n_samples)


#### Calculate KLD

In [16]:
kld_original = 0.5 * ((mu_y - mu_x) @ np.linalg.inv(cov_y) @ (mu_y - mu_x).T +
                      np.trace(np.linalg.inv(cov_y) @ cov_x) -
                      np.log(np.linalg.det(cov_x) / np.linalg.det(cov_y)) - d_dimensions)

print(f'KLD: {kld_original:.4f}')

KLD: 0.8000


### RBIG - KLD

In [19]:
X.min(), X.max()

(-4.006934109277744, 4.585027222023813)

In [20]:
Y.min(), Y.max()

(-4.607129910785054, 4.299322691460413)

In [17]:
%%time

n_layers = 100000
rotation_type = 'PCA'
random_state = 0
zero_tolerance = 60
tolerance = None
pdf_extension = 10
pdf_resolution = None
verbose = 0

# Initialize RBIG class
kld_rbig_model = RBIGKLD(n_layers=n_layers, 
                  rotation_type=rotation_type, 
                  random_state=random_state, 
                  zero_tolerance=zero_tolerance,
                  tolerance=tolerance,
                     pdf_resolution=pdf_resolution,
                    pdf_extension=pdf_extension,
                    verbose=verbose)

# fit model to the data
kld_rbig_model.fit(X, Y);

CPU times: user 5min 46s, sys: 10.9 ms, total: 5min 46s
Wall time: 12.4 s


In [18]:
# Save KLD value to data structure
kld_rbig= kld_rbig_model.kld*np.log(2)

print(f'KLD (RBIG): {kld_rbig:.4f}')
print(f'KLD: {kld_original:.4f}')

KLD (RBIG): 0.8349
KLD: 0.8000
